In [1]:
import os
import time
import shutil
import tkinter as tk
from datetime import datetime

returnal_folder = os.path.join(os.environ['USERPROFILE'], 'AppData\Local\Returnal\Steam\Saved\\')
backup_folder = returnal_folder+'Backup'


def check_current_saved():    
    file_path = os.path.join(returnal_folder, 'SaveGames', 'saveprofile.sav')
    if os.path.exists(file_path):
        mod_time = os.path.getmtime(file_path)
        mod_time_str = time.ctime(mod_time)
        mod_time_dt = datetime.fromtimestamp(mod_time)
        print(f"Current Returnal Saved Game is from {mod_time_str}.")
        return mod_time_dt
    else:
        print(f"The file {file_path} does not exist.")
        
def check_if_backup_folder_exists():
    if not os.path.exists(backup_folder):
        os.makedirs(backup_folder)
        print(f"Folder '{backup_folder}' created.")
    else:
        print(f"Folder '{backup_folder}' already exists.")
        
def copy_files(source_dir, dest_dir):
    files_to_copy = ['SaveProfile.sav', 'SaveProfile.susres.sav', 'SaveProfile.susresvalid.sav']

    for file in files_to_copy:
        source_path = os.path.join(source_dir, file)
        dest_path = os.path.join(dest_dir, file)
        shutil.copy(source_path, dest_path)
        print(f"File '{file}' copied to '{dest_dir}'")
        
        
# def latest_modified_dir(folder_path):
#     latest_dir = None
#     latest_time = 0
#     for root, dirs, files in os.walk(folder_path):
#         for dir in dirs:
#             dir_path = os.path.join(root, dir)
#             mod_time = os.path.getmtime(dir_path)
#             if mod_time > latest_time:
#                 latest_dir = dir_path
#                 latest_time = mod_time
#     if latest_dir is None:
#         print("No directories found in folder.")
#     else:
#         return latest_dir     
        
def save_current_game(current_saved_timestamp):
    timestamp = current_saved_timestamp.strftime("%Y-%m-%d_%H-%M-%S")
    folder_name = f"save_{timestamp}"
    folder_path = os.path.join(backup_folder, folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        message = f"Folder '{folder_name}' created. Game saved!"     

        source_dir = returnal_folder+'SaveGames'
        dest_dir = folder_path        
        copy_files(source_dir,dest_dir)         
    else:
        message = f"Folder '{folder_name}' already exists. Game not saved."

    return message

def load_saved_game(savegame_folder):
    if os.path.exists(savegame_folder):
        dest_dir = returnal_folder+'SaveGames'
        source_dir = savegame_folder
        copy_files(source_dir,dest_dir)
    else:
        print('error, no se pudo loadear')

In [ ]:
check_if_backup_folder_exists()

window = tk.Tk()
window.title("Returnal Load/Save v0.1")

# create the three buttons
save_button = tk.Button(window, text="Save Game",width=20, height=2)
load_button = tk.Button(window, text="Load Game",width=20, height=2)
exit_button = tk.Button(window, text="Exit")

# create a listbox to show the backup directories
backup_listbox = tk.Listbox(window,width=50, height=10)

# create a text output widget
output_text = tk.Text(window,width=50, height=10)

# define the function to be called when the save button is clicked
def save_game():
    current_saved_time = check_current_saved()
    save_current_game(current_saved_time)
    output_text.insert(tk.END, "Game saved successfully.\n")
    refresh_saved_games_list()

# define the function to be called when the load button is clicked
def load_game():
    selection = backup_listbox.curselection()
    if selection:
        index = selection[0]
        folder_name = backup_listbox.get(index)
        folder_path = os.path.join(backup_folder, folder_name)
        load_saved_game(folder_path)
        output_text.insert(tk.END, f"Game loaded from {folder_path}.\n")
    else:
        output_text.insert(tk.END, "Please select a backup folder to load.\n")

# define the function to be called when the exit button is clicked

def refresh_saved_games_list():
    # Clear the listbox
    backup_listbox.delete(0, tk.END)
    # Get the list of directories in the backup folder
    backup_folders = os.listdir(backup_folder)
    # Add each folder name to the listbox
    for folder_name in backup_folders:
        backup_listbox.insert(tk.END, folder_name)

def exit_app():
    window.destroy()

# add the buttons, listbox, and text output widget to the window
save_button.pack(side="top")
load_button.pack(side="top")
backup_listbox.pack()
output_text.pack()
exit_button.pack()

# bind the functions to the buttons
save_button.config(command=save_game)
load_button.config(command=load_game)
exit_button.config(command=exit_app)

# populate the backup_listbox with the directories in the backup_folder
backup_folders = os.listdir(backup_folder)
for folder_name in backup_folders:
    backup_listbox.insert(tk.END, folder_name)

# start the main event loop
window.mainloop()